https://edinburghcyclehire.com/
Souřadnnice sanic jsou uváděny na 14 respektive 16 desetinných míst (Decimal degrees in WGS84).



In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 500)
pd.set_option("display.precision", 14)

In [215]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from bokeh.io import output_notebook, show
from bokeh.models import WMTSTileSource
output_notebook()

Loading BokehJS ...

In [ ]:
# vyloučit 3 "jízdy" s end stanicí 280 (Exhibition Centre Liverpool)
rides = pd.read_csv("201809-202010.csv", parse_dates=[0,1], usecols = ["started_at","ended_at","duration","start_station_id", "end_station_id"])    

In [158]:
# Vytvoření tabulky obsahující stanice = načtení z csv a dopočítání vzdálenosti mezi stanicemi.
# Zdrojová data obsahují nekonzistence v názvech a popisech stanic. Jejich zeměpisná šířka a délka také není u všech údajů stejná. To přičítám odchylce GPS a různému umístění kola.
# Některé stanice jsou jen v konečné stanici (například stanice id=242). Vymazána koncová stanice id 280 - Exhibition Centre Liverpool (výstava kol).
# Celková tabulka stations je vytvořena spojením informací ze "start" a "end" slouců. Duplicity jsou vyřešeny vybráním prvního popisu a průměrem souřadnic.
# Načtení informací o "start" a "end" stanicích:
# Některé stanice jsou dočasné během akcí - zatím njsou dohromady s ostatními
stations = pd.read_csv("201809-202010.csv", index_col = "start_station_id", usecols = ["start_station_id","start_station_name","start_station_description","start_station_latitude", "start_station_longitude"])   
end_stations = pd.read_csv("201809-202010.csv", index_col = "end_station_id", usecols = ["end_station_id","end_station_name","end_station_description","end_station_latitude", "end_station_longitude"])  
# Přejmenování sloupců a indexu:
column_names = {"start_station_name":"name","start_station_description":"description","start_station_latitude":"latitude", "start_station_longitude":"longitude"}
stations.rename(columns=column_names, index={"start_station_id": "idd"}, inplace=True)
stations.index.names = ['id']
end_column_names = {"end_station_name":"name","end_station_description":"description","end_station_latitude":"latitude", "end_station_longitude":"longitude"}
end_stations.rename(columns=end_column_names, index={'end_station_id': 'id'}, inplace=True)
end_stations.index.names = ['id']
# Sloučení informací o "start" a "end" stanicích a jejich setřídění:
stations = stations.append(end_stations)
stations.sort_index(axis="index", inplace = True)
# Výpočet průměrných koorinátů a přidání k prvním názvům a popisům stanic:
stations_coord_mean = stations.groupby(["id"]).mean()
stations = stations.groupby(["id"]).first()
stations["latitude"] = stations_coord_mean["latitude"]
stations["longitude"] = stations_coord_mean["longitude"]
# Vymazána koncová stanice id 280 - Exhibition Centre Liverpool (výstava kol). Lepším řešením je vymazat "stanice", které jsou vzdáleny od Edinburgu víc než 50? km.
stations.drop(280, inplace=True)
# přepočet wgs84 souřadnic na  Mercator projekci pro použití v WMTS tile map v openstreetmap.org 
stations["mer_x"] = stations["longitude"] * (6378137 * np.pi/180.0)
stations["mer_y"] = np.log(np.tan((90 + stations["latitude"]) * np.pi/360.0)) * 6378137
# Vymazání dočasných df:
end_stations = end_stations[0:0]
stations_coord_mean = stations_coord_mean[0:0]

In [167]:
rides.loc[rides["end_station_id"]==280]

,started_at,ended_at,duration,start_station_id,end_station_id
27010,2019-03-22 12:15:48.326000+00:00,2019-03-22 13:24:21.301000+00:00,4112,349,280
27013,2019-03-22 12:44:51.585000+00:00,2019-03-23 07:41:04.990000+00:00,68173,357,280
27184,2019-03-23 13:20:22.230000+00:00,2019-03-23 14:19:19.087000+00:00,3536,189,280


In [216]:
# openstreetmap používá mercator coordináty
p = figure(plot_width=1200, plot_height=700, tools='pan, wheel_zoom, save', x_range=(-380000,-340000), y_range=(7545000,7553000), x_axis_type="mercator", y_axis_type="mercator") # Edinburg
p.add_tile(WMTSTileSource(url="https://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"))
p.circle(stations["mer_x"], stations["mer_y"], size=8, color="red", alpha=0.5)
show(p)